# Interactive Plot of Determinant function for varying wavelength

User can slide a bar to change input wavelength and see how this affects propagation constants for N1 fiber.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact, plotlogf
from ipywidgets import interactive, FloatSlider, Layout
from scipy.optimize import newton
from ngsolve.webgui import Draw

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [n_air, n_glass, n_air, n_glass, n_air]

mats = ['air', 'glass', 'air', 'glass', 'air']

maxhs = [1, 1, 1, 1, 1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs,
               no_mesh=True,
              )


# Get loss profile

In [ ]:
n = 500
wls = np.linspace(1.4e-6, 2e-6, n+1)
betas1 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

In [ ]:
for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, ns=ns, mats=mats, maxhs=maxhs, wl=wl)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low) +0*1j
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-12)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess = 1/.999999 * guess.real + imag * 1j
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag  # set current imaginary part as guess for next one
                flag=False
        except RuntimeError:
            guess = .999999 * guess.real + 1j * imag
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    betas1[i] = beta
    

In [ ]:
# Formula for loss spikes from article

ms = np.arange(11,15,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d = ts[1]

ls = (2 * n1 * d / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver
ls

In [ ]:
# Formula for loss spikes from article

ms = [16, 17, 18, 19, 20]

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d2 = ts[3]

ls2 = (2 * n1 * d2 / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver
ls2

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,5))
ax = plt.gca()

ax.plot(wls, -betas1.imag, color='green', linewidth=.9)
plt.yscale('log')
ym,yM = ax.get_ylim()
for l in ls:
    ax.plot([l,l], [0, yM],  linewidth=1, color='black', linestyle=':')
    
for l in ls2:
    ax.plot([l,l], [0, yM],  linewidth=1, color='orange', linestyle=':')

In [ ]:
ls2, ls

# Interactive plot

Click on the slider and drag to change wavelength.  Or you can click on the slider and then use the arrow keys on your keyboard to move it a step at a time

### Second spike set

Fundamental appears as first root near real axis near left of plot. It has already started to become lossy (ie move further below real axis).  

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [n_air, n_glass, n_air, n_glass, n_air]

mats = ['air', 'glass', 'air', 'glass', 'air']

maxhs = [1, 1, 1, 1, 1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs,
               no_mesh=True,
              )


In [ ]:
nu = 1
N = 75
Ktype = 'kappa'
cent = ls2[1]*1e6

def det_plot(wl):

    A.wavelength = wl * 1e-6
        
    cutoff = A.ks[0] * A.scale
    L, R = .99979, 1.00009
    imin, imax = -.03, .01
    Xr = np.linspace(L * cutoff , R * cutoff , num=N)
    Xi = np.linspace(imin, imax, num=N)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, figsize=(14, 8))
    
    fs1 = A.determinant(zs, nu=nu, outer='h2', Ktype=Ktype)
    data = ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=N)
    ax1.grid(True)
    ax1.set_facecolor('grey')
    ax1.set_yticks([0])
    ax1.set_xticks([cutoff], labels=['k_low'])
    plt.title('Current wavelength: %.6e\nResonant wavelength: %.6e'%(wl*1e-6, cent*1e-6), fontsize=16)
#     plt.colorbar(data)
#     plt.show()

interactive_plot = interactive(det_plot, wl=FloatSlider(min=cent - .001, 
                                                        max=cent + .003, 
                                                        step=.0001, 
                                                        value=cent - .001, 
                                                        readout_format='.6f',
                                                        continuous_update=True,
                                                        layout=Layout(width='100%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot 

# Mode profiles at secondary spike

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [n_air, n_glass, n_air, n_glass, n_air]

mats = ['air', 'glass', 'air', 'glass', 'air']

maxhs = [.1, .03, .1, .01, .1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs,
                wl=ls2[1],
              )
nu=1
outer='h2'

In [ ]:
k_low = A.k0 * A.ns[0] * A.scale

plotlogf(A.determinant, .99982*k_low, 1.00001*k_low, -.005,.001, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(.99982*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer)))


In [ ]:
Fs = A.all_fields(beta1, nu, outer)

In [ ]:
Draw(Fs['Ez'], A.mesh)

In [ ]:
Draw(Fs['Etv'].Norm(), A.mesh)


# Primary Spike, but not also secondary

Farthest right black vertical dashed line in plot below

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,5))
ax = plt.gca()

ax.plot(wls, -betas1.imag, color='green', linewidth=.9)
plt.yscale('log')
ym,yM = ax.get_ylim()
for l in ls:
    ax.plot([l,l], [0, yM],  linewidth=1, color='black', linestyle=':')
    
for l in ls2:
    ax.plot([l,l], [0, yM],  linewidth=1, color='orange', linestyle=':')

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [n_air, n_glass, n_air, n_glass, n_air]

mats = ['air', 'glass', 'air', 'glass', 'air']

maxhs = [1, 1, 1, 1, 1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs,
               no_mesh=True,
              )


In [ ]:
nu = 1
N = 75
Ktype = 'kappa'
cent = ls[0]*1e6

def det_plot(wl):

    A.wavelength = wl * 1e-6
        
    cutoff = A.ks[0] * A.scale
    L, R = .9997, 1.00009
    imin, imax = -.01, .01
    Xr = np.linspace(L * cutoff , R * cutoff , num=N)
    Xi = np.linspace(imin, imax, num=N)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, figsize=(14, 8))
    
    fs1 = A.determinant(zs, nu=nu, outer='h2', Ktype=Ktype)
    data = ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=N)
    ax1.grid(True)
    ax1.set_facecolor('grey')
    ax1.set_yticks([0])
    ax1.set_xticks([cutoff], labels=['k_low'])
    plt.title('Current wavelength: %.6e\nResonant wavelength: %.6e'%(wl*1e-6, cent*1e-6), fontsize=16)
#     plt.colorbar(data)
    plt.show()

interactive_plot = interactive(det_plot, wl=FloatSlider(min=cent - .001, 
                                                        max=cent + .003, 
                                                        step=.00005, 
                                                        value=cent - .001, 
                                                        readout_format='.6f',
                                                        layout=Layout(width='100%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot 

In [ ]:
nu = 1
N = 75
Ktype = 'kappa'
cent = ls[0]*1e6

def det_plot(wl):

    A.wavelength = wl * 1e-6
        
    cutoff = A.ks[0] * A.scale
    L, R = .9994, 1.00009
    imin, imax = -.02, .02
    Xr = np.linspace(L * cutoff , R * cutoff , num=N)
    Xi = np.linspace(imin, imax, num=N)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, figsize=(14, 8))
    
    fs1 = A.determinant(zs, nu=nu, outer='h2', Ktype=Ktype)
    data = ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=N)
    ax1.grid(True)
    ax1.set_facecolor('grey')
    ax1.set_yticks([0])
    ax1.set_xticks([cutoff], labels=['k_low'])
    plt.title('Current wavelength: %.6e\nResonant wavelength: %.6e'%(wl*1e-6, cent*1e-6), fontsize=16)
#     plt.colorbar(data)
    plt.show()

interactive_plot = interactive(det_plot, wl=FloatSlider(min=cent - .001, 
                                                        max=cent + .003, 
                                                        step=.00005, 
                                                        value=cent - .001, 
                                                        readout_format='.6f',
                                                        layout=Layout(width='100%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot 

# Mode profiles at primary (but not secondary) spike

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [n_air, n_glass, n_air, n_glass, n_air]

mats = ['air', 'glass', 'air', 'glass', 'air']

maxhs = [.1, .02, .1, .02, .1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs,
                wl=ls[0],
              )
nu=1
outer='h2'

In [ ]:
k_low = A.k0 * A.ns[0] * A.scale

plotlogf(A.determinant, .99982*k_low, 1.00001*k_low, -.01,.01, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(.99996*k_low)

beta1 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer)))


In [ ]:
Fs = A.all_fields(beta1, nu, outer)

## Z-Components

In [ ]:
Draw(Fs['Ez'], A.mesh)

In [ ]:
Draw(Fs['Etv'].real, A.mesh)

# Previous fundamental?

In [ ]:
guess = np.array(.9999*k_low)

beta2 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta2, ". Residual of determinant: ", abs(A.determinant(beta2, nu, outer)))


In [ ]:
Fs2 = A.all_fields(beta2, nu, outer)

## Z-Components

In [ ]:
Draw(Fs2['Ez'], A.mesh)

In [ ]:
Draw(Fs2['Etv'].real, A.mesh)

# Primary and Secondary Spike (where ls overlap)

4th spike from right in plot below

In [ ]:
%matplotlib inline
fig = plt.figure(figsize=(8,5))
ax = plt.gca()

ax.plot(wls, -betas1.imag, color='green', linewidth=.9)
plt.yscale('log')
ym,yM = ax.get_ylim()
for l in ls:
    ax.plot([l,l], [0, yM],  linewidth=1, color='black', linestyle=':')
    
for l in ls2:
    ax.plot([l,l], [0, yM],  linewidth=1, color='orange', linestyle=':')

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [n_air, n_glass, n_air, n_glass, n_air]

mats = ['air', 'glass', 'air', 'glass', 'air']

maxhs = [1, 1, 1, 1, 1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs,
               no_mesh=True,
              )


In [ ]:
nu = 1
N = 75
Ktype = 'kappa'
cent = ls[1]*1e6

def det_plot(wl):

    A.wavelength = wl * 1e-6
        
    cutoff = A.ks[0] * A.scale
    L, R = .99982, 1.00009
    imin, imax = -.01, .01
    Xr = np.linspace(L * cutoff , R * cutoff , num=N)
    Xi = np.linspace(imin, imax, num=N)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, figsize=(14, 8))
    
    fs1 = A.determinant(zs, nu=nu, outer='h2', Ktype=Ktype)
    data = ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=N)
    ax1.grid(True)
    ax1.set_facecolor('grey')
    ax1.set_yticks([0])
    ax1.set_xticks([cutoff], labels=['k_low'])
    plt.title('Current wavelength: %.6e\nResonant wavelength: %.6e'%(wl*1e-6, cent*1e-6), fontsize=16)
#     plt.colorbar(data)
    plt.show()

interactive_plot = interactive(det_plot, wl=FloatSlider(min=cent - .003, 
                                                        max=cent + .002, 
                                                        step=.00005, 
                                                        value=cent - 0., 
                                                        readout_format='.6f',
                                                        layout=Layout(width='100%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot 

# Mode profiles at combined spike

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [n_air, n_glass, n_air, n_glass, n_air]

mats = ['air', 'glass', 'air', 'glass', 'air']

maxhs = [.1, .02, .1, .01, .1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs,
                wl=1.72385e-6,
              )
nu=1
outer='h2'

In [ ]:
k_low = A.k0 * A.ns[0] * A.scale

plotlogf(A.determinant, .99982*k_low, 1.00001*k_low, -.01,.01, nu, outer,
         iref=100, rref=100, levels=100)

In [ ]:
guess = np.array(182.29-.0025j)

beta1 = newton(A.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(A.determinant(beta1, nu, outer)))


In [ ]:
Fs = A.all_fields(beta1, nu, outer)

## Z-Components

In [ ]:
Draw(Fs['Ez'], A.mesh)

In [ ]:
Draw(1e-2*Fs['Etv'].Norm(), A.mesh)